In [2]:
# setup

import tensorflow as tf
import tensorflow.keras
import keras_tuner as kt
keras = tensorflow.keras

In [ ]:
# from tensorflow import keras

In [3]:
# Download and prepare the dataset

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


In [4]:
# define hypermodel
# can use a model builder func or subclass HyperModel

def model_builder(hp: kt.HyperParameters):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # tune num units in first dense layer [32, 512]
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # tune learning rate for optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    return model
    

In [5]:
# create tuner and find best model

tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10,
                     factor=3, directory='my_dir', project_name='intro_to_kt')
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(img_train, label_train, epochs=50,
             validation_split=0.2, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 30 Complete [00h 00m 41s]
val_accuracy: 0.8872500061988831

Best val_accuracy So Far: 0.8895000219345093
Total elapsed time: 00h 11m 38s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [6]:
# build model with optimal hyperparameters and train
model: keras.Model = tuner.hypermodel.build(best_hps)
history: keras.callbacks.History = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))

print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4942 - accuracy: 0.8251 - val_loss: 0.4052 - val_accuracy: 0.8527
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3710 - accuracy: 0.8633 - val_loss: 0.3796 - val_accuracy: 0.8584
Epoch 3/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3317 - accuracy: 0.8783 - val_loss: 0.3715 - val_accuracy: 0.8643
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3056 - accuracy: 0.8866 - val_loss: 0.3347 - val_accuracy: 0.8777
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2860 - accuracy: 0.8928 - val_loss: 0.3476 - val_accuracy: 0.8767
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2717 - accuracy: 0.8984 - val_loss: 0.3496 - val_accuracy: 0.8749
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2590 - accuracy: 0.9041 - val_loss: 0.3371 - val_accuracy:

In [7]:
type(history)

keras.callbacks.History